In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from tec_an import *
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *
from bitstamp import *

from sklearn.utils import shuffle

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

In [3]:
load_from_disk = True

# '2Min'

In [4]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [5]:

def getModel():
    param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
    #param = {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 3, 'random_state': 42}
    coModel = RandomForestClassifier(**param)
    return coModel

In [6]:
%%time

def get_classifiers():
    clss = classifiers = [
        #lambda : MLPClassifier(alpha=1,
        #                       solver = 'lbfgs', 
        #                       max_iter=1000000, 
        #                       random_state = 42),
        #lambda : MLPClassifier(alpha=1, max_iter=10000, random_state = 42),
        
        
        lambda : make_pipeline( StandardScaler(),
        MLPClassifier(
            solver='lbfgs', alpha=1, random_state=1, max_iter=20000,
            early_stopping=True)
        ),
        
        lambda : xgb.XGBClassifier(random_state=1,
                                   objective = "binary:logistic",
                                   eval_metric='mlogloss',
                                   learning_rate=0.01, 
                                   use_label_encoder = True),
        
        #lambda : GaussianNB(),
        lambda : QuadraticDiscriminantAnalysis(),
        #lambda : AdaBoostClassifier(random_state = 42),
        #lambda : CatBoostClassifier(logging_level = 'Silent'),
        
        
        #lambda : LogisticRegression(),
        #lambda : KNeighborsClassifier(3),
        #lambda : GaussianProcessClassifier(1.0 * RBF(1.0)),
        
        
        #lambda : DecisionTreeClassifier(random_state = 42),
        #lambda : RandomForestClassifier(max_depth=50, n_estimators=100, max_features=1),
        
        #lambda : SVC(gamma=2, C=1, random_state = 42),
    ]
    
    def rand():
        ##print("rand created")
        param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
        return RandomForestClassifier(**param)
    

    #clss = []
    clss.append(lambda : rand())
    return clss


estimators = [
                #RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=6),
                lambda : RFE(estimator=RandomForestClassifier(random_state=10), n_features_to_select=10),
                lambda : RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=8),
                lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10)),
                lambda : RFECV(estimator=RandomForestClassifier(random_state=10))
                ]


def get_all_models():
    models = []
    classifiers = get_classifiers()
    for cls in classifiers:
        models.append(cls())
        #models.append(MockModel(cls()))
        #models.append(MockCoPilotModel(cls(), getModel()))
        
        for est in estimators:
            tmp = lambda : Pipeline(
                steps=[
                ('s',est()),
                ('m',cls())
            ])
            models.append(tmp())
            #models.append(MockModel(tmp()))
            #models.append(MockCoPilotModel(tmp(), getModel()))
    return models


CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11 µs


In [7]:
get_all_models_factory = lambda : get_all_models()

In [8]:
%%time

def test_models(provider, get_all_models_factory, steps = [1]):
    
    score_board_by_step = {}

    for step in steps:
        print(f"Step {step}")
        models_list = get_all_models_factory()
        model_result = []
        for idx in tqdm(range(len(models_list))):
            model_creator = models_list[idx]
            model = model_creator
            #print(f"Trainning {model}")
            result = run_trial(model_creator, provider, step)

            result['step'] = step

            model_result.append(result)
        
        score_board_by_step[step] = model_result
        
    best_models = []

    best_model = []

    best_score = -1
    
    best_step = -1
    
    model_rank = []
    
    def myFunc(e):
        return e['profit']

    for step_idx in score_board_by_step:
        step_board = score_board_by_step[step_idx]
        for result in step_board:
            if (result['profit'] > 0):
                model_rank.append(result)
            
    model_rank.sort(key=myFunc, reverse = True)
    
    return model_rank

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 9.78 µs


In [9]:

steps = [1, 3]

windows = [5, 10, 20, 30, 40]


In [10]:

def test_windows(minutes, steps = steps):
    results = []
    for win in tqdm(windows):
        tec = TecAn(windows = win, windows_limit = 100)
        sourceDataGenerator = SourceDataGenerator(tec = tec)
        online = get_online_data(minutes, sourceDataGenerator, load_from_disk, win)
        result = test_models(
            online, 
            get_all_models_factory,
            steps = steps
        )
        for rank in result:
            rank['window'] = win
            rank['minutes'] = minutes
            
        results.append(result)
    return results
    

In [11]:
%%time

minutes = [15, 5, 3]

min_results = []

for minu in minutes:
    print(f"Evaluating {minu} min")
    results = test_windows(minu, steps)
    min_results.append(results)


Evaluating 15 min


  0%|          | 0/5 [00:00<?, ?it/s]

Step 1


  0%|          | 0/20 [00:00<?, ?it/s]

[23:45:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:46:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[23:46:07] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameter

...

Step 3


  0%|          | 0/20 [00:00<?, ?it/s]

[07:29:04] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[07:29:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[07:29:27] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:516: 
Parameters: { use_label_encoder } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameter

/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/samirantonio/opt/miniconda3/envs/py3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


CPU times: user 12h 51min 7s, sys: 1h 3min 52s, total: 13h 54min 59s
Wall time: 8h 10s


In [28]:
model_path = f'model/all_results'
dump(min_results, model_path)

['model/all_results']

In [54]:
best_results = []
for results in min_results:
    for r in results:
        if (len(r) > 0):
            for win_result in r:
                profit = win_result['profit']
                model = win_result['model']
                window = win_result['window']
                minutes = win_result['minutes']
                step = win_result['step']
                profit = win_result['profit']
                if (profit > 103):
                    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
                    #print(f"{win_result['model']}")
                    print(f"")
                    best_results.append(win_result)
        else:
            print("Not found")

print(f"{len(best_results)} selected")

Minutes=15 Window=5 Step=3 | 105.15566999872978

Minutes=15 Window=5 Step=1 | 104.50015039617749

Minutes=15 Window=5 Step=3 | 104.36058823335532

Minutes=15 Window=5 Step=3 | 103.04717581292306

Minutes=15 Window=5 Step=3 | 103.04717581292306

Minutes=15 Window=20 Step=3 | 104.98789310098186

Minutes=15 Window=20 Step=1 | 104.20696559102844

Minutes=15 Window=20 Step=3 | 103.90494600389799

Minutes=15 Window=20 Step=1 | 103.1940586063731

Minutes=15 Window=30 Step=1 | 106.15136273262588

Minutes=15 Window=30 Step=1 | 104.2910618304422

Minutes=15 Window=30 Step=3 | 103.69295833355675

Minutes=15 Window=40 Step=1 | 103.103646042194

Minutes=5 Window=30 Step=3 | 104.74088450062459

Minutes=5 Window=30 Step=3 | 104.21861576725837

Minutes=3 Window=5 Step=3 | 103.09810839002479

Minutes=3 Window=20 Step=3 | 103.11000442357648

17 selected


In [31]:
model_path = f'model/results'
dump(best_results, model_path)

['model/results']

In [32]:
def myFunc(e):
    return e['profit']

best_results.sort(key=myFunc, reverse = False)

for best in best_results:
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    profit = best['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
    print(f"{model}")
    model_path = f'model/result_{window}_{minutes}_{step}'
    dump(best, model_path)
    print(f"")

Minutes=15 Window=5 Step=3 | 103.04717581292306
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.01, max_delta_step=0,
              max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, random_state=1, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              use_label_encoder=True, validate_parameters=1, verbosity=None)

Minutes=15 Window=5 Step=3 | 103.04717581292306
Pipeline(steps=[('s', RFECV(estimator=DecisionTreeClassifier(random_state=10))),
                ('m',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                          

In [33]:
def myFunc(e):
    return e['profit']

best_results.sort(key=myFunc, reverse = True)



In [34]:
best_results[0]

{'profit': 106.15136273262588,
 'model': RandomForestClassifier(criterion='entropy', max_depth=100, n_estimators=30,
                        random_state=42),
 'models_profit': {'btcusd': 6.1514},
 'models_score': {'btcusd': {'recall': 0.06217616580310881,
   'precision': 0.26666666666666666,
   'f1': 0.10084033613445378,
   'accuracy': 0.7857857857857858,
   'roc_auc': 0.5106166188817033}},
 'models_profit_metric': {'btcusd': 0.08180782064359458},
 'step': 1,
 'window': 30,
 'minutes': 15}

In [35]:
def load_online(minutes, window, val_end = 1626388860, currency = "btcusd"):
    tec = TecAn(windows = window, windows_limit = 100)
    source_data_generator = SourceDataGenerator(tec = tec)


    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 40,
                 val_limit = 999,
                 val_keys = [currency],
                 #val_start = val_start,
                 val_end = val_end,
                 train_start_list = []
    )

    online.load_cache()
    return online

In [36]:
def backtest_model(model, x, closed_prices, back):
    
    for idx in range(len(x)):
        xx = [x[idx]]
        yy = model.predict(xx)[0]
        price = closed_prices[idx]
        #print(f'{idx} {yy} {price}')
        back.on_state(0, price)
        if(yy == 1):
            back.request_buy(price)
        else:
            back.request_sell(price)
    
    print(f"Closing backtest_model {price} {back.current}")
    back.sell(price)
    print(f"Closed backtest_model {back.current}")
    #back.request_sell(31744.62)
    
    return back

In [37]:
def test_model(model, set_key, provider, step, verbose = True):
    valX, valY = provider.load_val_data(set_key)

    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    #print(len(x))

    preds = model.predict(x)

    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    f1 = f1_score(y, preds)
    accuracy = accuracy_score(y, preds)

    back = BackTest(value = 100, 
                    verbose = verbose, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    back = backtest_model(model, x, closed_prices, back)
    return back

In [42]:
from datetime import datetime


timestamp = int(datetime.timestamp((datetime.now())))


#timestamp = 1610959352

In [ ]:
online_cache = {}

In [48]:
scoreboard = []

fails = []

currency = "btcusd"


for best in best_results:
    model = best['model']
    window = best['window']
    minutes = best['minutes']
    step = best['step']
    profit = best['profit']
    print(f"Minutes={minutes} Window={window} Step={step} | {profit}")
    cache_key = f"{minutes}-{window}"
    try:
        online = online_cache[cache_key]
    except :
        print(f"Not found {cache_key}")
        online = load_online(minutes = minutes, window = window, val_end = timestamp, currency = currency)
        online_cache[cache_key] = online
        
    try:
        back = test_model(model, currency, online, step, False)
        score = {}
        score['profit'] = back.get_profit()
        score['result'] = best
        scoreboard.append(score)
    except :
        fails.append(best)


Minutes=15 Window=30 Step=1 | 106.15136273262588
Closing backtest_model 31630.0 101.95310952189699
Closed backtest_model 101.95310952189699
Minutes=15 Window=5 Step=3 | 105.15566999872978
Closing backtest_model 31630.0 103.78439988167241
Closed backtest_model 103.78439988167241
Minutes=15 Window=20 Step=3 | 104.98789310098186
Closing backtest_model 31630.0 102.44324306004856
Closed backtest_model 102.44324306004856
Minutes=5 Window=30 Step=3 | 104.74088450062459
Closing backtest_model 31630.0 98.76324984417592
Closed backtest_model 98.76324984417592
Minutes=15 Window=5 Step=1 | 104.50015039617749
Closing backtest_model 31630.0 99.60754178021709
Closed backtest_model 99.60754178021709
Minutes=15 Window=5 Step=3 | 104.36058823335532
Closing backtest_model 31630.0 106.41918622679833
Closed backtest_model 106.41918622679833
Minutes=15 Window=30 Step=1 | 104.2910618304422
Closing backtest_model 31630.0 101.42372010450366
Closed backtest_model 101.42372010450366
Minutes=5 Window=30 Step=3 | 

In [49]:
scoreboard.sort(key=myFunc, reverse = True)

In [53]:
profs = []
for score in scoreboard:
    profit = score['profit']
    result = score['result']
    profs.append(back.get_profit())
    print(f"{profit} \n{result}")

    
print()
print(f"{np.average(profs)} ")

6.4192 
{'profit': 104.36058823335532, 'model': Pipeline(steps=[('s', RFECV(estimator=DecisionTreeClassifier(random_state=10))),
                ('m',
                 Pipeline(steps=[('standardscaler', StandardScaler()),
                                 ('mlpclassifier',
                                  MLPClassifier(alpha=1, early_stopping=True,
                                                max_iter=20000, random_state=1,
                                                solver='lbfgs'))]))]), 'models_profit': {'btcusd': 4.3606}, 'models_score': {'btcusd': {'recall': 0.46099290780141844, 'precision': 0.43333333333333335, 'f1': 0.44673539518900346, 'accuracy': 0.5165165165165165, 'roc_auc': 0.5091422872340426}}, 'models_profit_metric': {'btcusd': 0.05864789474958374}, 'step': 3, 'window': 5, 'minutes': 15}
4.5586 
{'profit': 103.90494600389799, 'model': Pipeline(steps=[('s',
                 RFE(estimator=DecisionTreeClassifier(random_state=10),
                     n_features_to_sel